In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
import newspaper

In [2]:
driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")

/var/folders/px/4r2jc78x1h748q29m317jc6w0000gn/T/ipykernel_56275/1162476086.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")


In [3]:
#Settings
think_tank = "AEI"
start_date = "2022" #Stop scraping articles that are from 2022 or older.

In [4]:
df = pd.DataFrame(columns= ["THINK_TANK", "AUTHOR", "TITLE", "CONTENT", "DATE", "LINK", "TYPE"])
df = df[["TITLE", "AUTHOR", "CONTENT", "DATE", "TYPE", "THINK_TANK", "LINK"]]
df.head()

,TITLE,AUTHOR,CONTENT,DATE,TYPE,THINK_TANK,LINK


In [6]:
#Newspaper library settings
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
config = newspaper.Config()
config.browser_user_agent = user_agent

In [7]:
#Scrapes the title, date, content, authors of an article given the URL.
#Returns False if article is before the given start_date. returns True if article within date range, indicating that scraping should continue
#Using Both Newspaper and BeautifulSoup to scrape certain parts of the article.
 
def scrape_article(URL):
    page = requests.get(URL)
    soup = bs(page.content, "html.parser") #Using BeautifulSoup to download article
    
    try: #Using newspaper to download the article
        article = newspaper.Article(URL, config=config)
        article.download()
        article.parse()
    except:
        return True

    content = ""
    title = ""
    author = ""
    date = ""
    type = ""

    try:
        date = soup.find("p", class_="date").text
    except: pass
    
    if start_date in date:
        return False #indicates that no more pages should be scraped as it's out of the date range
    else:
        #Tries to gather all the information.
        try:
            content = article.text
        except:pass
        try:
            title = soup.find("h1", class_="entry-title").text
        except: pass
        try:
            author = ', '.join(article.authors)
        except: pass
        try:
            type = soup.find("span", class_="type").text.replace("[|]", "")
        except: pass
    
        row = [title, author, content, date, type, "AEI", URL]
        df.loc[len(df.index)] = row #Adds the article/publication to the dataframe as a row
        return True


In [8]:
#Run through all publication sites until start_date reached.
driver.get("https://www.aei.org/search-results/?wpsolr_fq%5B0%5D=tags_str:Climate%20change&wpsolr_page=1") #Overview of articles
soup = bs(driver.page_source, "html.parser")

run = True #is true until the first article before start_date is reached. 

while run:
    #Scrapes all articles on the overview page
    soup = bs(driver.page_source, "html.parser")

    search_content = soup.find("div", class_="col-12 col-lg-8 col-xl-9 results-list")

    for post in search_content.findAll("div", class_="row gy-3"):
        link = post.find("h4",  class_="title").find("a")["href"]
        run = scrape_article(link)
    
    #Goes to next page when all articles are scraped
    next_page = soup.find("a", class_="page-link",string="Next")
    if next_page is None:
        run = False
    else:
        driver.get(next_page["href"])
        soup = bs(driver.page_source, "html.parser")


In [9]:
df.head(5)

,TITLE,AUTHOR,CONTENT,DATE,TYPE,THINK_TANK,LINK
0,From Environmentalism to Climate Catastrophism...,Ruy Teixeira,Key Points\n\nThe environmental movement has a...,"May 10, 2023",Report,AEI,https://www.aei.org/research-products/report/f...
1,Letter: Electric Vehicles Won’t Save the Climate,Benjamin Zycher,Note: This letter to the editor appeared in th...,"April 26, 2023",Article,AEI,https://www.aei.org/articles/letter-electric-v...
2,Earth Day 2023: Utterly Bereft of Ideas,Benjamin Zycher,Earth Day falls on April 22 — Lenin’s birthday...,"April 21, 2023",Article,AEI,https://www.aei.org/articles/earth-day-2023-ut...
3,OPEC Remains an Obstacle for Biden’s Green-Ene...,"Benjamin Zycher, James Pethokoukis",Can it surprise anyone that the multiple Biden...,"April 17, 2023",Article,AEI,https://www.aei.org/articles/opec-remains-an-o...
4,Comment to the Department of Energy: Proposed ...,Benjamin Zycher,Summary\n\nThe estimates on benefits and costs...,"April 14, 2023",Testimony,AEI,https://www.aei.org/research-products/testimon...


In [10]:
df.to_csv("data/AEI.csv",encoding="utf-8-sig",index=False)